In [1]:
import numpy as np
import get_pdbinfo


In [14]:


def get_RW(fn, inpro):
    '''
    Select the HOH in [2.0, 3.5] of protein

    '''

    outfile = open("test/RW_info.txt","w")

    pro = get_pdbinfo.pdbinfo(fn,file = inpro)
    pro_atoms = pro.getPolarAtoms()
    protein,waters = get_pdbinfo.pdbinfo(fn, lines = pro_atoms).getProteinWaters()
    waters_coord = get_pdbinfo.pdbinfo(fn,lines = waters).getCoords()
    protein_coord = get_pdbinfo.pdbinfo(fn,lines = protein).getCoords()
    ### calculate distance ###
    waters_coord = np.expand_dims(waters_coord, 1)
    protein_coord = np.expand_dims(protein_coord,0)
    distance = np.linalg.norm(waters_coord - protein_coord, axis = 2)
    distance_min = np.min(distance, axis = 1)
    rw_index = []
    for idx, i in enumerate(distance_min):
        if i >= 2.0 and i <= 3.5:
            rw_index.append(idx)
    for i in rw_index:
        rw_line = waters[i]
        rw_distance = distance[i]
        rw_chain = get_pdbinfo.chid(rw_line)
        if rw_chain != " ":
            rw_name = str(int(get_pdbinfo.resi(rw_line))) + "." + get_pdbinfo.chid(rw_line)
        else:
            rw_name = str(int(get_pdbinfo.resi(rw_line)))
            
        for idx, d in enumerate(rw_distance):
            if d <= 3.5 and d >= 2.0:
                pro_idx = idx
                pro_line = protein[idx]
                pro_chain = get_pdbinfo.chid(pro_line)
                pro_name = get_pdbinfo.resn(pro_line)
                if pro_chain != " ":
                    pro_idx = str(int(get_pdbinfo.resi(pro_line))) + "." + pro_chain 
                else:
                    pro_idx = str(int(get_pdbinfo.resi(pro_line)))
                pro_aname = get_pdbinfo.atmn(pro_line).strip()
                outline = fn + "," + pro_name + "," + pro_idx + "," + pro_aname + "," + rw_name + "," + str(round(d,2)) + "\n"
                outfile.write(outline)
    outfile.close()
    return None

In [15]:
fn = "3c2f"
inpro = "test/3c2f_protein_all_SF_type2.pdb"
get_RW(fn, inpro)

2274
80
[13.283 -4.868 29.215]
[ 20.522 -17.258  15.117]
(80, 1, 3)
(1, 2274, 3)
20.116382005718624


In [42]:
np.sqrt((13.283-19.833)**2 + (-4.868+18.887)**2 + (29.215-13.446)**2)

22.092899809667358

In [20]:
def get_angle(a,b,c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

In [62]:
def get_BW(fn,water,lig):
    outfile = open("test/BW_info.txt","w")

    lig = get_pdbinfo.pdbinfo(fn,file = lig)
    lig = lig.getPolarAtoms()
    pro = get_pdbinfo.pdbinfo(fn,file = water)
    pro_atoms = pro.getPolarAtoms()
    protein,waters = get_pdbinfo.pdbinfo(fn, lines = pro_atoms).getProteinWaters()

    waters_coord_init = get_pdbinfo.pdbinfo(fn,lines = waters).getCoords()
    protein_coord_init = get_pdbinfo.pdbinfo(fn,lines = protein).getCoords()
    ligand_coord_init = get_pdbinfo.pdbinfo(fn,lines=lig).getCoords()

    ### select the HOH in 3.5 of ligand/protein ###
    waters_coord = np.expand_dims(waters_coord_init, 1)
    protein_coord = np.expand_dims(protein_coord_init,0)
    ligand_coord = np.expand_dims(ligand_coord_init,0)

    distance_pw = np.linalg.norm(waters_coord - protein_coord, axis = 2)
    distance_lw = np.linalg.norm(waters_coord - ligand_coord, axis = 2)
    distance_pw_min = np.min(distance_pw, axis = 1)
    distance_lw_min = np.min(distance_lw, axis = 1)

    bw_index = []
    for idx, (i1,i2) in enumerate(zip(distance_pw_min,distance_lw_min)):
        if i1 >= 2.0 and i1 <= 3.5 and i2 >= 2.0 and i2 <= 3.5:
            bw_index.append(idx)
    
    for idx in bw_index:
        bw_coord = waters_coord_init[idx]
        bw_distance_p = distance_pw[idx]
        bw_distance_l = distance_lw[idx]
        for num1, i1 in enumerate(bw_distance_p):
            if i1 >= 2.0 and i1 <= 3.5:
                p_coord = protein_coord_init[num1]
                for num2, i2 in enumerate(bw_distance_l):
                    if i2 >= 2.0 and i2 <= 3.5:
                        l_coord = ligand_coord_init[num2]
                        print(l_coord)
                        print(bw_coord)
                        angle = get_angle(p_coord, bw_coord, l_coord)
                        if angle >= 60:
                            print(angle)
                            bw_line = waters[idx]
                            bw_chain = get_pdbinfo.chid(bw_line)
                            if bw_chain != " ":
                                bw_name = str(int(get_pdbinfo.resi(bw_line))) + "." + get_pdbinfo.chid(bw_line)
                            else:
                                bw_name = str(int(get_pdbinfo.resi(bw_line)))
                            pro_line = protein[num1]
                            lig_line = lig[num2]
                            pro_chain = get_pdbinfo.chid(pro_line)
                            pro_name = get_pdbinfo.resn(pro_line)
                            if pro_chain != " ":
                                pro_idx = str(int(get_pdbinfo.resi(pro_line))) + "." + pro_chain 
                            else:
                                pro_idx = str(int(get_pdbinfo.resi(pro_line)))
                            pro_aname = get_pdbinfo.atmn(pro_line).strip()
                            lig_name = get_pdbinfo.atmn(lig_line).strip()
                            outline = fn + "," + pro_name + "," + pro_idx + "," + pro_aname + "," + bw_name + "," + lig_name + "," + str(round(i1,2)) + "," + str(round(i2,2)) + "," + str(int(round(angle))) + "\n"
                            outfile.write(outline)
    outfile.close()

In [63]:
fn = "3c2f"
inpro = "test/3c2f_protein_all.pdb"
inlig = "test/3c2f_ligand.pdb"
get_BW(fn,inpro,inlig)

[ 27.464 -17.006  44.905]
[ 30.117 -19.127  45.719]
80.61192497936587
[ 27.464 -17.006  44.905]
[ 30.117 -19.127  45.719]
117.31404695998768
[ 27.464 -17.006  44.905]
[ 30.117 -19.127  45.719]
65.32549527156894
[ 27.464 -17.006  44.905]
[ 30.117 -19.127  45.719]
106.83641041134564


In [55]:
np.sqrt((27.464-30.117)**2 + (-17.006 +19.127)**2 + (44.905-45.719)**2)

3.492799163994404

In [15]:
import sys
import os
import rdkit
from rdkit import Chem
from software_path import path_obabel
import get_pdbinfo
obabel = path_obabel()
def get_input(datadir, fn):
    olddir = os.getcwd()
    os.chdir(datadir)
    ### get ligand ###
    ### input should be provided as either of sdf file (best choice) or mol2 file ###
    inlig1 = fn + "_ligand.mol2"
    inlig2 = fn + "_ligand.sdf"
    inlig3 = fn + "_ligand.pdb"

    ### check ligand input file ###
    if inlig2 not in os.listdir("."):
        inlig = inlig1
        mol = Chem.MolFromMol2File(inlig,removeHs = False)
        if mol == None:
            error_massage = "Error:input ligand (mol2) should be checked\ntry use sdf as input format(make sure the atom type, bond connection are right"
            sys.exit(error_massage)
        else:
            inlig_rdkit = inlig1
    else:
        inlig = inlig2
        mol = Chem.SDMolSupplier(inlig,removeHs = False)[0]
        if mol == None:
            sys.exit("Error:input ligand (sdf) should be checked")
        else:
            inlig_rdkit = inlig2

    if inlig3 not in os.listdir("."):
        inlig = inlig_rdkit
        infmt = inlig.split(".")[-1]
        outlig = inlig3
        cmd = obabel + " -i" + infmt + " " +  inlig + " -opdb -O " + outlig
        os.system(cmd)    

    ### get protein ###
    ### at least one protein structure should be provided with all waters ###
    inpro1 = fn + "_protein.pdb"
    inpro2 = fn + "_protein_all.pdb"

    if inpro1 not in os.listdir("."):
        inpro = inpro2
        outpro = open(inpro1,"w")
        protein_lines = get_pdbinfo.pdbinfo(fn, file = inpro).getProteinWaters()[0]
        
        outpro.write("".join(protein_lines))
        outpro.close()

    ### check input structures ###
    if os.path.isfile(inlig_rdkit) and os.stat(inlig_rdkit).st_size != 0:
        print("Ligand for conformation stability:" + inlig_rdkit)
    else:
        sys.exit("Error: rdkit input")
    if os.path.isfile(inlig3) and os.stat(inlig3).st_size != 0:
        print("Ligand for Vina, SASA, BA, ION:" + inlig3)  
    else:
        sys.exit("Error: ligand input (pdb)")
    if os.path.isfile(inpro1) and os.stat(inpro1).st_size != 0:
        print("Protein without water molecules:" + inpro1)
    else:
        sys.exit("Error: protein input without water")
    if os.path.isfile(inpro2) and os.stat(inpro2).st_size != 0:
        print("Protein with water molecules:" + inpro2)
    else:
        sys.exit("Error: protein input with water")
    
    os.chdir(olddir)

    return inlig_rdkit, inlig3, inpro1, inpro2

In [18]:
datadir = "/Users/jianinglu1/Documents/script/deltaXGB_linux/Feature/test/"
fn = "4q90"
inlig_rdkit, inlig_pdb, inpro1, inpro2 = get_input(datadir, fn)

Ligand for conformation stability:4q90_ligand.mol2
Ligand for Vina, SASA, BA, ION:4q90_ligand.pdb
Protein without water molecules:4q90_protein.pdb
Protein with water molecules:4q90_protein_all.pdb
